# IMPLEMENTATION OF CNN IN TENSORFLOW

Let's develop better intuition for how Convolutional Neural Networks (CNN) work. We'll examine
how humans classify images, and then see how CNNs use similar approaches.

In [1]:
import tensorflow as tf
# Output depth
k_output = 64
# Image Properties
image_width = 10
image_height = 10
color_channels = 3
# Convolution filter
filter_size_width = 5
filter_size_height = 5
# Input/Image
input = tf.placeholder(
tf.float32,
shape=[None, image_height, image_width, color_channels])
# Weight and bias
weight = tf.Variable(tf.truncated_normal(
[filter_size_height, filter_size_width, color_channels, k_output]))
bias = tf.Variable(tf.zeros(k_output))
# Apply Convolution
conv_layer = tf.nn.conv2d(input, weight, strides=[1, 2, 2, 1], padding='SAME')
# Add bias
conv_layer = tf.nn.bias_add(conv_layer, bias)
# Apply activation function
conv_layer = tf.nn.relu(conv_layer)

C:\Python\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
conv_layer = tf.nn.conv2d(input, weight, strides=[1, 2, 2, 1], padding='SAME')
conv_layer = tf.nn.bias_add(conv_layer, bias)
conv_layer = tf.nn.relu(conv_layer)
# Apply Max Pooling
conv_layer = tf.nn.max_pool(
conv_layer,
ksize=[1, 2, 2, 1],
strides=[1, 2, 2, 1],
padding='SAME')

# DATASETS

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)
import tensorflow as tf
# Parameters
learning_rate = 0.00001
epochs = 1
batch_size = 128
# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256
# Network Parameters
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting .\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting .\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting .\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting .\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
# Store layers weight & bias
weights = {
'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
'out': tf.Variable(tf.random_normal([1024, n_classes]))}
biases = {
'bc1': tf.Variable(tf.random_normal([32])),
'bc2': tf.Variable(tf.random_normal([64])),
'bd1': tf.Variable(tf.random_normal([1024])),
'out': tf.Variable(tf.random_normal([n_classes]))}

# CONVOLUTIONS

In [6]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [5]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [7]:
def conv_net(x, weights, biases, dropout):
# Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
# Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
# Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
# Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# TESTING ACCURACY/RESULTS

In [9]:
# tf Graph input

x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)
# Model
logits = conv_net(x, weights, biases, keep_prob)
# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)
# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# Initializing the variables
init = tf. global_variables_initializer()
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})
# Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})
            print('Epoch {:>2}, Batch {:>3} -'
                'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
            epoch + 1,
            batch + 1,
            loss,
            valid_acc))
# Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 49919.9336 Validation Accuracy: 0.125000
Epoch  1, Batch   2 -Loss: 37933.8516 Validation Accuracy: 0.148438
Epoch  1, Batch   3 -Loss: 29749.4395 Validation Accuracy: 0.164062
Epoch  1, Batch   4 -Loss: 28978.5234 Validation Accuracy: 0.160156
Epoch  1, Batch   5 -Loss: 26227.9727 Validation Accuracy: 0.171875
Epoch  1, Batch   6 -Loss: 23310.8281 Validation Accuracy: 0.164062
Epoch  1, Batch   7 -Loss: 22305.2715 Validation Accuracy: 0.179688
Epoch  1, Batch   8 -Loss: 23357.2617 Validation Accuracy: 0.171875
Epoch  1, Batch   9 -Loss: 19099.4883 Validation Accuracy: 0.171875
Epoch  1, Batch  10 -Loss: 20197.6660 Validation Accuracy: 0.191406
Epoch  1, Batch  11 -Loss: 19542.8750 Validation Accuracy: 0.187500
Epoch  1, Batch  12 -Loss: 21019.9297 Validation Accuracy: 0.175781
Epoch  1, Batch  13 -Loss: 19555.2852 Validation Accuracy: 0.171875
Epoch  1, Batch  14 -Loss: 21729.5430 Validation Accuracy: 0.164062
Epoch  1, Batch  15 -Loss: 17093.7930 Validation

Epoch  1, Batch 122 -Loss:  5116.7720 Validation Accuracy: 0.609375
Epoch  1, Batch 123 -Loss:  3436.7012 Validation Accuracy: 0.613281
Epoch  1, Batch 124 -Loss:  3204.3989 Validation Accuracy: 0.609375
Epoch  1, Batch 125 -Loss:  4998.8203 Validation Accuracy: 0.617188
Epoch  1, Batch 126 -Loss:  5870.1826 Validation Accuracy: 0.621094
Epoch  1, Batch 127 -Loss:  6285.7925 Validation Accuracy: 0.636719
Epoch  1, Batch 128 -Loss:  4001.6714 Validation Accuracy: 0.632812
Epoch  1, Batch 129 -Loss:  4092.8430 Validation Accuracy: 0.628906
Epoch  1, Batch 130 -Loss:  4818.8208 Validation Accuracy: 0.628906
Epoch  1, Batch 131 -Loss:  3884.9307 Validation Accuracy: 0.632812
Epoch  1, Batch 132 -Loss:  5340.4902 Validation Accuracy: 0.625000
Epoch  1, Batch 133 -Loss:  4615.8467 Validation Accuracy: 0.621094
Epoch  1, Batch 134 -Loss:  3806.3633 Validation Accuracy: 0.617188
Epoch  1, Batch 135 -Loss:  3947.0366 Validation Accuracy: 0.621094
Epoch  1, Batch 136 -Loss:  4170.1816 Validation

Epoch  1, Batch 243 -Loss:  2671.0732 Validation Accuracy: 0.703125
Epoch  1, Batch 244 -Loss:  2672.9893 Validation Accuracy: 0.707031
Epoch  1, Batch 245 -Loss:  2519.7837 Validation Accuracy: 0.699219
Epoch  1, Batch 246 -Loss:  2388.9316 Validation Accuracy: 0.703125
Epoch  1, Batch 247 -Loss:  2532.7957 Validation Accuracy: 0.710938
Epoch  1, Batch 248 -Loss:  2622.2739 Validation Accuracy: 0.718750
Epoch  1, Batch 249 -Loss:  2185.5598 Validation Accuracy: 0.726562
Epoch  1, Batch 250 -Loss:  2459.4487 Validation Accuracy: 0.710938
Epoch  1, Batch 251 -Loss:  2719.8506 Validation Accuracy: 0.722656
Epoch  1, Batch 252 -Loss:  2295.7671 Validation Accuracy: 0.726562
Epoch  1, Batch 253 -Loss:  1981.0527 Validation Accuracy: 0.718750
Epoch  1, Batch 254 -Loss:  2304.9658 Validation Accuracy: 0.722656
Epoch  1, Batch 255 -Loss:  2074.7124 Validation Accuracy: 0.722656
Epoch  1, Batch 256 -Loss:  2690.4863 Validation Accuracy: 0.714844
Epoch  1, Batch 257 -Loss:  3053.3667 Validation

Epoch  1, Batch 364 -Loss:  2140.2930 Validation Accuracy: 0.699219
Epoch  1, Batch 365 -Loss:  2628.9990 Validation Accuracy: 0.707031
Epoch  1, Batch 366 -Loss:  1678.2190 Validation Accuracy: 0.710938
Epoch  1, Batch 367 -Loss:  1917.7104 Validation Accuracy: 0.710938
Epoch  1, Batch 368 -Loss:  1931.7659 Validation Accuracy: 0.707031
Epoch  1, Batch 369 -Loss:  1586.5148 Validation Accuracy: 0.707031
Epoch  1, Batch 370 -Loss:  1423.8953 Validation Accuracy: 0.707031
Epoch  1, Batch 371 -Loss:  2086.1450 Validation Accuracy: 0.707031
Epoch  1, Batch 372 -Loss:  2236.9971 Validation Accuracy: 0.703125
Epoch  1, Batch 373 -Loss:  1950.5381 Validation Accuracy: 0.703125
Epoch  1, Batch 374 -Loss:  2693.3289 Validation Accuracy: 0.707031
Epoch  1, Batch 375 -Loss:  2661.0176 Validation Accuracy: 0.714844
Epoch  1, Batch 376 -Loss:  2150.5991 Validation Accuracy: 0.714844
Epoch  1, Batch 377 -Loss:  1796.4341 Validation Accuracy: 0.710938
Epoch  1, Batch 378 -Loss:  1823.5459 Validation

We can further improve the accuracy by increasing the epochs.